In [9]:
import http.client
import json
import datetime
import pandas as pd
from statistics import mean
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from sklearn import linear_model

# Utils OpenCovid

In [10]:
def CasesPer100K(inputNumber):
    if inputNumber <= 4:
        return 1
    elif inputNumber <= 9:
        return 2
    elif inputNumber <= 50:
        return 3
    elif inputNumber <= 100:
        return 4
    elif inputNumber <= 199:
        return 5
    else:
        return 6
def ConfirmedDeaths100K(inputNumber):
    if inputNumber < 0.1: 
        return 2
    elif inputNumber <= 1:
        return 3
    elif inputNumber <= 2:
        return 4
    elif inputNumber <= 5:
        return 5
    else:
        return 6
def PositivityRate(inputNumber):
    if inputNumber <= 2.9:
        return 1
    elif inputNumber <= 4.9:
        return 2
    elif inputNumber <= 7.9:
        return 3
    elif inputNumber <= 10:
        return 4
    elif inputNumber <= 15:
        return 5
    else:
        return 6
def Tests100K(inputNumber):
    if inputNumber >= 5000:
        return 1
    elif inputNumber >= 3000:
        return 2
    elif inputNumber >= 2000:
        return 3
    elif inputNumber >= 1000:
        return 4
    elif inputNumber >= 500:
        return 5
    else:
        return 6
def BedsOccupied(inputNumber):
    if inputNumber <= 3:
        return 1
    elif inputNumber <= 7:
        return 2
    elif inputNumber <= 12:
        return 3
    elif inputNumber <= 15:
        return 4
    elif inputNumber <= 20:
        return 5
    else:
        return 6
def BedsUCIOccupied(inputNumber):
    if inputNumber <= 3:
        return 1
    elif inputNumber <= 7:
        return 2
    elif inputNumber <= 12:
        return 3
    elif inputNumber <= 15:
        return 4
    elif inputNumber <= 20:
        return 5
    else:
        return 6
def ConvertToScale(inputNumber, target):
    if target == 'confirmedCases100k':
        return CasesPer100K(inputNumber)
    elif target == 'appliedTests100k':
        return Tests100K(inputNumber)
    elif target == 'positivityRate':
        return PositivityRate(inputNumber)
    elif target == 'bedsOccupiedPercentage':
        return BedsOccupied(inputNumber)
    elif target == 'bedsUCIOccupiedPercentage':
        return BedsUCIOccupied(inputNumber)

# Utils Environmental

In [11]:
def getFactor(element, difference):
    if element == 'pm10':
        sigDif = int(difference/10)
        return (1.76/100)*sigDif
    elif element == 'pm2.5':
        sigDif = int(difference/10)
        return (2.24/100)*sigDif
    elif element == 'o3':
        sigDif = int(difference/10)
        return (4.76/100)*sigDif

# Utils Vaccine

In [22]:
def getVaccineEffect (dose, brand, risk_2):
    if dose == 0:
        return risk_2
    
    elif dose == 1:
        if brand == 'PFIZER':
            return risk_2 * (1 - 0.78)
        elif brand == 'ASTRAZENECA':
            return risk_2 * (1 - 0.6401)
        elif brand == 'SINOPHARM':
            return risk_2 * (1 - 0.68397)
        else:
            return risk_2 * ((1 - 0.68397) + (1 - 0.6401) + (1 - 0.78))/3
        
    elif dose == 2:
        if brand == 'PFIZER':
            return risk_2 * (1 - 0.95)
        elif brand == 'ASTRAZENECA':
            return risk_2 * (1 - 0.704)
        elif brand == 'SINOPHARM':
            return risk_2 * (1 - 0.79)
        else:
            return risk_2 * ( (1 - 0.79) + (1 - 0.704) + (1 - 0.95))/3
    

# Fetching and general functions

In [12]:
def getMonday(dateNow):
    today = dateNow
    today = today + datetime.timedelta(days=-today.weekday(), weeks=-1)
    return today.strftime("%d-%m-%Y")

def getOffSetTime():
    x = datetime.date(2021,9,29)
    return int((x - (datetime.date.today())).days/7)

def getOpenCovidPeruData(date):
    mondayDate = getMonday(date)
    conn = http.client.HTTPSConnection("open-covid-api-vwgk4ckqbq-uk.a.run.app")
    payload = ''
    headers = {}
    conn.request("GET", f"/api/semaforo?fecha={mondayDate}", payload, headers)
    res = conn.getresponse()
    data = res.read()
    jsonData = json.loads(data.decode('utf-8'))
    filteredData =  list(map((lambda value: {'region':value['region'],'appliedTests':value['avgTest'],'testPositivityPercentage':value['positividad'],'population':value['poblacion'],'bedsOccupiedPercentage':value['camasCovid'],'bedsUCIOccupiedPercentage':value['uci'],'confirmedCasesWeekly':(value['poblacion']*value['incid_100'])/100000,'deathsCasesWeekly':(value['poblacion']*value['fall_100'])/100000}), jsonData[0]['regions']))
    df = pd.DataFrame(filteredData)
    return df

def getProcessedTable(date):
    data = getOpenCovidPeruData(date)
    regions = data['region']
    confirmedDeaths100k = (data['deathsCasesWeekly'] / data['population']) * 10**6
    confirmedCases100k = (data['confirmedCasesWeekly'] / data['population']) * 10**6
    appliedTests100k = (data['appliedTests'] / data['population']) * 10**6
    positivityRate = (data['testPositivityPercentage'])
    bedsOccupiedPercentage = (data['bedsOccupiedPercentage'])
    bedsUCIOccupiedPercentage = data['bedsUCIOccupiedPercentage']
    finalDF = pd.concat([regions,confirmedDeaths100k,confirmedCases100k,appliedTests100k,positivityRate,bedsOccupiedPercentage,bedsUCIOccupiedPercentage],axis=1,join='inner',keys=['regions','confirmedDeaths100k', 'confirmedCases100k','appliedTests100k','positivityRate','bedsOccupiedPercentage','bedsUCIOccupiedPercentage'])
    finalDF = finalDF.round(2)
    return finalDF

def getIndex(city):
    endDate = 196 + getOffSetTime()
    rangeDates = list(range(endDate,0,-7))
    totalIndividual = ''
    currentDF = '' 
    for i in rangeDates:
        try:
            testDf = getProcessedTable(datetime.date.today()-datetime.timedelta(days=i))
            testDf = testDf[testDf['regions'] == city]
            del testDf['regions']

            if i == rangeDates[-1]:
                currentDF = testDf

            if i == endDate:
                totalIndividual = testDf
            else:
                totalIndividual = pd.concat([totalIndividual,testDf],ignore_index=True)
        except:
            continue

    correlation_matrix = totalIndividual.corr()
    correlation_matrix_filtered = correlation_matrix['confirmedDeaths100k']
    correlation_matrix_filtered = correlation_matrix_filtered[[1,2,3,4,5]]
    correlation_matrix_filtered = correlation_matrix_filtered/sum(correlation_matrix_filtered)
    correlation_matrix_filtered = pd.DataFrame(correlation_matrix_filtered)
    correlation_matrix_filtered.columns = [city]
    
    weights_table_city = correlation_matrix_filtered
    weights_table_city = weights_table_city.transpose()
    
    totalRisk = 0
    for rate in ['confirmedCases100k','appliedTests100k','positivityRate','bedsOccupiedPercentage','bedsUCIOccupiedPercentage']:
        dataRate = float(currentDF[rate])
        riskNoWeight = ConvertToScale(dataRate,rate)
        factorWeight = float(weights_table_city[rate])
        riskWeighted = factorWeight * riskNoWeight
        totalRisk = totalRisk + riskWeighted
    
    return totalRisk

def getWaqiData():
    pm10Total = []
    pm25Total = []
    o3Total = []
    
    pm10AvgTotal = []
    pm25AvgTotal =[]
    o3AvgTotal = []
    
    for department in ['@7580','@7577','@7578','@8780','@379']:
        conn = http.client.HTTPSConnection("api.waqi.info")
        payload = ''
        headers = {}
        conn.request("GET", f"/feed/{department}/?token=d91a4fcc7a848a548eaaeba48ae3a30f08e5d402", payload, headers)
        res = conn.getresponse()
        data = res.read()
        jsonData = json.loads(data.decode('utf-8'))
        jsonDataFiltered = jsonData['data']['iaqi']
        jsonDataAvgFiltered = jsonData['data']['forecast']['daily']
        
        for unit in ['pm10','pm25','o3']:
            if unit in jsonDataFiltered.keys() and unit == 'pm10':
                pm10Total.append(jsonDataFiltered[unit]['v'])
            if unit in jsonDataFiltered.keys() and unit == 'pm25':
                pm25Total.append(jsonDataFiltered[unit]['v'])
            if unit in jsonDataFiltered.keys() and unit == 'o3':
                o3Total.append(jsonDataFiltered[unit]['v'])
                
            if unit in jsonDataAvgFiltered.keys():
                unitData = [] 
                for day in jsonDataAvgFiltered[unit]:
                    avg = day['avg']
                    unitData.append(avg)
                if unit == 'pm10':
                    pm10AvgTotal.append(mean(unitData))
                if unit == 'pm25':
                    pm25AvgTotal.append(mean(unitData))
                if unit == 'o3':
                    o3AvgTotal.append(mean(unitData))
                    
        
    pm10 = mean(pm10Total)
    pm25 = mean(pm25Total)
    o3 = mean(o3Total)
    
    
    pm10Avg = mean(pm10AvgTotal)
    pm25Avg = mean(pm25AvgTotal)
    o3Avg = mean(o3AvgTotal)
    
    currentData = {'pm10':pm10,'pm2.5':pm25,'o3':o3}
    avgData = {'pm10':pm10Avg, 'pm2.5':pm25Avg,'o3':o3Avg}
    
    return {'currentData':currentData,'avgData':avgData}

def getRisk2nd(inputNumber):
    environmentalData = getWaqiData()
    increments = []
    for element in ['pm10','pm2.5','o3']:
        difference = abs(environmentalData['currentData'][element] - environmentalData['avgData'][element])
        factor = getFactor(element,difference)
        increments.append(factor * inputNumber)
    returnNumber = inputNumber
    for increment in increments:
        returnNumber = returnNumber + increment
    return returnNumber

# Final function

In [23]:
def getRisk3ndComplete(city, dose, brand):
    first = getIndex(city)
    second = getRisk2nd(first)
    third = getVaccineEffect (dose, brand, second)
    return third

for city in ['LIMA METROPOLITANA']:
    print(f'{city}: ', getRisk3ndComplete(city, 1, 'ASTRAZENECA'))

LIMA METROPOLITANA:  1.9506570823820257
